<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ufidon/nlp/blob/main/asrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ufidon/nlp/blob/main/asrs.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>
<br>

# [Speech Recognition & Synthesis Applications](https://huggingface.co/learn/audio-course/chapter7/introduction)
- From [HuggingFace Audio Course](https://huggingface.co/learn/audio-course)

- 📖 [Unit 4: Build A Music Genre Classifier](https://huggingface.co/learn/audio-course/chapter4)

In [ ]:
# ①. Pre-trained models and datasets for audio classification
# install the latest transformers
!pip install git+https://github.com/huggingface/transformers

# Keyword spotting (KWS) is the task of identifying a keyword in a spoken utterance. 

In [ ]:
# 1.1. load dataset Minds-14
from datasets import load_dataset

minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")

In [ ]:
# 1.2. load the classifier
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14",
)

In [ ]:
# 1.3. try a sample
classifier(minds[0]["audio"])

In [ ]:
# 2. Speech Commands
# load dataset of speech commands
speech_commands = load_dataset(
    "speech_commands", "v0.02", split="validation", streaming=True
)
sample = next(iter(speech_commands))

In [ ]:
# load a classifier for speech commands
classifier = pipeline(
    "audio-classification", model="MIT/ast-finetuned-speech-commands-v2"
)
classifier(sample["audio"].copy())

In [ ]:
# verify the result
from IPython.display import Audio

Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [ ]:
# 3. Language Identification
# FLEURS (Few-shot Learning Evaluation of Universal Representations of Speech) 
# is a dataset for evaluating speech recognition systems in 102 languages

# load up a sample from the validation split of the FLEURS dataset using streaming mode:
fleurs = load_dataset("google/fleurs", "all", split="validation", streaming=True)
sample = next(iter(fleurs))

In [ ]:
# load a classifier for language identification
classifier = pipeline(
    "audio-classification", model="sanchit-gandhi/whisper-medium-fleurs-lang-id"
)

In [ ]:
# try it on the sample
classifier(sample["audio"])

In [ ]:
# 4. Zero-Shot Audio Classification
# Load the Environmental Speech Challenge (ESC) dataset:
dataset = load_dataset("ashraq/esc50", split="train", streaming=True)
audio_sample = next(iter(dataset))["audio"]["array"]

# define our candidate labels form the set of possible classification labels. 
candidate_labels = ["Sound of a dog", "Sound of vacuum cleaner"]

# run both through the model to find the candidate label 
# that is most similar to the audio input:
classifier = pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)
classifier(audio_sample, candidate_labels=candidate_labels)

In [ ]:
# Confirm the result
Audio(audio_sample, rate=16000)

In [ ]:
# ② Fine-tuning a model for music classification
# 1. Load the GTZAN dataset
# https://huggingface.co/datasets/marsyas/gtzan
# a popular dataset of 1,000 songs for music genre classification. 
# Each song is a 30-second clip from one of 10 genres of music, 
# spanning disco to metal.
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

In [ ]:
# split GTZAN
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

In [ ]:
# taste an example
# the sampling rate is 22,050 Hz
gtzan["train"][0]

In [ ]:
# genre id to label
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

In [ ]:
# 1.2 listen to a few more examples by using Gradio
import gradio as gr


def generate_audio():
    example = gtzan["train"].shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"],
    ), id2label_fn(example["genre"])


with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(4):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

demo.launch(debug=True)

In [ ]:
# 2. Picking a pretrained model for audio classification
# DistilHuBERT is a much smaller (or distilled) version of the HuBERT model
# https://huggingface.co/ntu-spml/distilhubert
# https://arxiv.org/abs/2110.01900
# https://huggingface.co/docs/transformers/model_doc/hubert


In [ ]:
# 3. From audio to machine learning features
# 3.1 Preprocessing the data
# instantiating the feature extractor for DistilHuBERT
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

In [ ]:
# model sampling rate
sampling_rate = feature_extractor.sampling_rate
sampling_rate

In [ ]:
# adjust data sampling rate
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [ ]:
# check the sampling rate conversion
gtzan["train"][0]

In [ ]:
# 3.2 feature scaling
# taste: compute the mean and variance of our first audio sample
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

In [ ]:
# apply the feature extractor
# The `input_values` are the preprocessed audio inputs 
#   that we’d pass to the HuBERT model. 
# The `attention_mask` is used when we process a batch of 
# audio inputs at once - it is used to tell the model 
# where we have padded inputs of different lengths.
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

In [ ]:
# 3.3 apply preprocess to the whole dataset
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [ ]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

In [ ]:
# rename the genre column to label:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

# obtain the label mappings from the dataset.
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

In [ ]:
# 4. fine-tune the model
# 4.1 create a model for fine-tuning
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

In [ ]:
# 4.2 define the training arguments
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"./",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
)

In [ ]:
# 4.3 define the metrics
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
# 4.4 instantiate the Trainer
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [ ]:
# launch the training
trainer.train()

In [ ]:
#
from transformers import pipeline

pipe = pipeline(
    "audio-classification", model="./checkpoint-570"
)

In [ ]:
gtzan["train"][0]['audio']

In [ ]:
# # you may need to install https://github.com/cisco/openh264
pipe(gtzan["train"][0]['audio'])

In [ ]:
pipe('test.wav')

In [ ]:
# 5. Build a demo with Gradio
from transformers import pipeline

# model_id = "./checkpoint-570"
# pipe = pipeline("audio-classification", model=model_id)

def classify_audio(filepath):
    preds = pipe(filepath)
    outputs = {}
    for p in preds:
        outputs[p["label"]] = p["score"]
    return outputs

import gradio as gr

demo = gr.Interface(
    fn=classify_audio, inputs=gr.Audio(type="filepath"), outputs=gr.Label()
)
demo.launch(debug=True)

### 📖 [Unit 7: Putting It All Together](https://huggingface.co/learn/audio-course/chapter7)
- 🎏 Speech-to-speech translation: 
  - translate speech from one language into speech in a different language

- 🎏 Creating a voice assistant: 
  - build your own voice assistant that works in a similar way to Alexa or Siri

- 🎏 Transcribing meetings: 
  - transcribe a meeting and label the transcript with who spoke when

### 📖 [Unit 3: Transformer Architectures For Audio](https://huggingface.co/learn/audio-course/chapter3)